In [1]:
import mlflow
import pandas as pd
from mlflow import MlflowClient

In [12]:
REMOTE_TRACKING_URI = "http://ec2-44-217-145-52.compute-1.amazonaws.com:5000"
experiment_name = "base_pattern"
mlflow.set_tracking_uri(REMOTE_TRACKING_URI)
client = MlflowClient(REMOTE_TRACKING_URI)
dataset = "criteo"

experiment = mlflow.get_experiment_by_name(experiment_name)
mlflow.set_experiment(experiment_id=experiment.experiment_id)

<Experiment: artifact_location='s3://kaggle-nakayama/mlflow/6', creation_time=1730853199485, experiment_id='6', last_update_time=1730853199485, lifecycle_stage='active', name='base_pattern', tags={}>

In [28]:
runs = mlflow.search_runs(experiment_ids=experiment.experiment_id)
name_flg = runs["tags.mlflow.runName"].str.contains(dataset)
tag_flg = runs["tags.package"] == "causalml"
no_parent_flg = runs["tags.mlflow.parentRunId"].isna()
run_df = runs.loc[name_flg & tag_flg & no_parent_flg]

In [30]:
with mlflow.start_run(run_name=f"{dataset}", tags={"dataset": dataset}) as parent_run:
    pass

2024/11/23 09:08:39 INFO mlflow.tracking._tracking_service.client: 🏃 View run criteo at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/6/runs/a511dbc6c6194232a264044ec5866e8f.
2024/11/23 09:08:39 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://ec2-44-217-145-52.compute-1.amazonaws.com:5000/#/experiments/6.


In [31]:
for run_id in run_df.run_id:
    run = client.set_tag(run_id, "mlflow.parentRunId", parent_run.info.run_id)